# 기본 설정

구글 드라이브 마운트

패키지 설치

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install transformers
!pip install datasets
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os
import tqdm
from transformers import ( #transformer에서 3가지의 모델을 알고 있으면 된다. #AUTO는 쉽게 사용할 수 있게 support 해주는 것
    AutoTokenizer, #텍스트 입력을 숫자 데이터로 변환해주는 것
    AutoModel, #모델 관련 (아키텍쳐 그 자체)
    DataCollatorWithPadding #각 시퀀스의 길이가 다를 수 있기 때문에, 길이의 통일성을 맞춤
)

from datasets import Dataset, Value
import torch
import torch.nn as nn
from typing import List, Optional, Tuple, Union
import datetime
import re

In [ ]:
#@title 마이너 패키지 로딩
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

<ipython-input-4-c31c0f4e8541>:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [ ]:
cd /gdrive/MyDrive/Lectures/2023/RecSys/content-based

/gdrive/MyDrive/Lectures/2023/RecSys/content-based


# 데이터 로드

`../raw/ml-100k`
- u1.base: 학습 (80K)
- u1.test: 검증 (20K)

In [ ]:
#@title get_user_item_map
def get_user_item_map(X):
  """Function to generate a ratings matrx and mappings for
  the user and item ids to the row and column indices

  Parameters
  ----------
  X : pandas.DataFrame, shape=(n_ratings,>=3)
      First 3 columns must be in order of user, item, rating.

  Returns
  -------
  user_map : pandas Series, shape=(n_users,)
      Mapping from the original user id to an integer in the range [0,n_users)
  item_map : pandas Series, shape=(n_items,)
      Mapping from the original item id to an integer in the range [0,n_items)
  """
  user_col, item_col, rating_col = X.columns[:3]
  rating = X[rating_col]
  user_map = pd.Series(
      index=np.unique(X[user_col]),
      data=np.arange(X[user_col].nunique()),
      name='user_map',
  )
  item_map = pd.Series(
      index=np.unique(X[item_col]),
      data=np.arange(X[item_col].nunique()),
      name='columns_map',
  )

  return user_map, item_map

In [ ]:
item_plot_df = pd.read_csv('movie_plots_80_missings.csv', index_col=0)

In [ ]:
def load_data(file_path):
  ratings_df = pd.read_csv(file_path, sep='\t', header=None,
                          names=['userId', 'movieId', 'rating', 'timestamp'])
  ratings_df['timestamp'] = ratings_df['timestamp'].apply(datetime.fromtimestamp)
  ratings_df = ratings_df.sort_values('timestamp')
  return ratings_df

In [ ]:
train_df = load_data('../raw/ml-100k/u1.base').reset_index(drop=True)
val_df = load_data('../raw/ml-100k/u1.test').reset_index(drop=True) #검증 데이터 셋

In [ ]:
user_map, item_map = get_user_item_map(pd.concat((train_df, val_df), axis=0))

In [ ]:
train_df['plots'] = train_df.movieId.map(item_plot_df['plot']) #외부에서 만든 무비 item plot을 train_df에 넣어줌
val_df['plots'] = val_df.movieId.map(item_plot_df['plot'])
#외부 데이터에서 긁어온 item의 plot 내용을 넣어줘서 새로운 데이터 프레임을 만듦

# 모델링

컨텐츠 모델링
- 플롯(plot) 활용: Distilbert (https://huggingface.co/docs/transformers/model_doc/distilbert#distilbert)

유저 모델링
- 단순 임베딩

결합
- 연결: concatenation

예측
- Linear regression

아이템 플롯 데이터
- movie_plots_80_missings.csv

참고
- ~~https://ratsgo.github.io/nlpbook/docs/lm~~
- https://huggingface.co/docs/transformers/index


In [ ]:
#huggingface의 식별자 로딩
# 사전 훈련된 모델 = Distilbert
pretrained_model_tag = "distilbert-base-uncased"

## 데이터셋

In [ ]:
#토크나이저를 초기화하여 로딩 'distilbert' 관련 토크나이저를 만들어줌
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_tag)
#하나의 객체로 반환됨

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["plots"], truncation=True, max_length=256)
#plot 줄거리 텍스트를 숫자 벡터로 변환해줌 (시퀀스의 길이 256으로 만듦)

데이터셋 변환
- `Dataset.map(fn)`

In [ ]:
def make_dataset(df):
    ds = Dataset.from_pandas(pd.concat([df.userId.map(user_map),
                              df.movieId.map(item_map),
                              df.rating,
                              df.plots],
                             axis=1)
                            )
    ds = ds.map(preprocess_function, batched=True)
    #preprocess_fucntion : plot을 숫자 벡터로 바꾸는 일 수행 (토크나이저 불러옴) -> input_ids가 생성됨
    ds = ds.rename_column("userId", "user_id")
    ds = ds.rename_column("movieId", "movie_id")
    ds = ds.rename_column("rating", "label")
    ds = ds.cast_column('label', Value("float"))
    return ds.remove_columns(['plots', 'movie_id']) #movieID는 필요 X

토크나이징

- 토크나이징 : Text 데이터 셋을 연속된 숫자 벡터로 변환해주는 것
- attention_mask : 해당 단어의 중요도를 지정해주는 것
- 토크나이징을 통해 텍스트를 토큰으로 변환한 후, 이를 임베딩 벡터나 정수 인덱스로 변환

In [ ]:
train_ds = make_dataset(train_df)
validation_ds = make_dataset(val_df)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/20000 [00:00<?, ? examples/s]

포맷 변경
- `torch` 포맷으로...

In [ ]:
train_ds = train_ds.with_format('torch').shuffle( #user.id, movie.id, plot으로 이루어짐
        seed=0
    )
validation_ds = validation_ds.with_format('torch')

## DataCollatorWithPadding

모델에 전달할 최종 입력과 출력 가공
- 만약 입력이 B개의 미니 배치로 구성될 때
- 각 토큰 시퀀스인 `input_ids`의 길이가 다르다면
- 끝을 패딩하여, 미니 배치 내의 모든 시퀀스의 길이가 동일하게 해주는 역할
- (필요시 출력도 변경...)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### DataCollator 테스트

In [ ]:
inputs = train_ds[:16] #16개의 input을 가져옴

inputs_len = [len(x) for x in inputs['input_ids']]
#길이가 256이 되도록 토크나이저를 진행했기에 대부분의 길이가 256임

# 입력 시퀀스의 길이 조사
print(inputs_len)

# 문장들을 배치로 묶고 패딩합니다.
# 배치가 딕셔너리 구조처럼 생성 ex) {"user_ids" : , "input_ids" : ,'attention_mask' : }
# minibatch size = 16    (16,256)모양의 텐서처럼 생성
batch = data_collator(inputs)

# 결과를 출력합니다.
print(batch['input_ids'][15])
# 인덱스 15인 값 : 110 -> 110만큼만 값이 존재, 나머지 146은 값이 없기 때문에 0으로 채워줌
# 길이가 110인 시퀀스가 256으로 길이가 바뀌게 된다. null 값은 0으로 패딩되어 길이를 256으로 맞춤


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[210, 256, 256, 256, 210, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 110]
tensor([  101,  1037,  2047,  2259,  2103,  2522,  6491, 18903, 10727,  1010,
        20706,  2245,  2000,  2022,  1037,  2671,  3836,  1010,  2003,  3253,
         1037,  3105,  2000,  6570,  1996,  2336,  1997,  2019,  2789,  2647,
        21237,  1012,  1037, 17935,  4588,  2937,  1999,  2637,  2003, 20706,
         2245,  2000,  2022,  2019,  3834,  3836,  2011,  1037,  4387,  1997,
         2019,  2789,  2647, 21237,  1012,  2016,  2003,  4778,  2000,  2037,
         2406,  2006,  2008, 13534,  6772,  1998,  2003,  2356,  2000,  2022,
         1996, 14924,  1997,  1996, 21237,  1005,  1055,  2336,  1012,  2096,
         2045,  1010,  2016,  5363,  2000,  2530,  4697,  1996,  2878,  2406,
         1012,  1024,  1024,  4116,  2022,  8024,  7245,  1026, 14863,  8024,
         7245,  1030, 16260,  1012,  6187,  6895,  1012,  4012,  1028,   102,
            0,     0,     0,     0,     0,     0,     0,     

In [ ]:
batch[input_ids].shape

# 모델 정의

모델 설정과 모델 클래스 생성
- DistilBertForRegressionConfig
- DistilBertForRegression

In [ ]:
#DistilBERT 모델을 위한 새로운 설정 클래스
from transformers import DistilBertConfig
from transformers.modeling_outputs import SequenceClassifierOutput
#가장 전형적인 것
from transformers.models.distilbert.modeling_distilbert import DistilBertModel
#transformer 모델을 거대 말뭉치로 학습시킨 것 (사전학습된 것)
from transformers.models.distilbert.modeling_distilbert import DistilBertPreTrainedModel

# DistilBertConfig를 상속받아 새로운 설정 클래스 정의
class DistilBertForRegressionConfig(DistilBertConfig):
    def __init__(self, user_num, factor_num, regressor_dropout, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.user_num = user_num
        self.factor_num = factor_num #user imbedding vector 길이
        self.regressor_dropout = regressor_dropout #최종적으로 linear regression에 dropout을 얼마의 확률로 걸지를 결정

# DistilBertModel 바디에 Regression 헤드를 추가한 클래스 정의 -  사전학습 모델을 상속받음
class DistilBertForRegression(DistilBertPreTrainedModel):
    config_class = DistilBertForRegressionConfig

    def __init__(self, config):
        super().__init__(config)

        # 유저 임베딩 레이어
        self.embed_user = nn.Embedding(config.user_num, config.factor_num)
        # 모델 바디
        self.distilbert = DistilBertModel(config)
        # 예측 헤드
        self.regressor_dropout = nn.Dropout(config.regressor_dropout)
        self.regressor = nn.Linear(config.hidden_size + config.factor_num, config.num_labels)

        # 가중치를 로드하고 초기화
        self.init_weights()

    #핵심적인 것 !!! 모델의 순전파 과정을 의미
    def forward(self, user_id=None, input_ids=None, attention_mask=None,
                #attention_mask : 토크나이저가 만든 것을 그대로 통과시키는 역할
                labels=None, **kwargs):
        # 유저 ID를 통해 생성된 유저 임베딩 벡터 생성
        embed_user_output = self.embed_user(user_id)

        # distilbert 블랙박스 통과 과정
        # distilbert 출력 생성하고, 맨 앞 토큰의 히든 임베딩을 조회
        distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask,
                               **kwargs) #**kwargs : keyword argument (확장시킨 것)
        pooled_output = distilbert_output.last_hidden_state[:, 0]
        #distilbert의 가장 마지막 출력층 = last_hidden_state 의 0번째 index 값을 가져옴
        # 출력 : 3차원 Tensor로 구성 (B, S, D) = (Batch Size = 16, Sequence = 256, D=768)

        # 유저 임베딩과 히든 임베딩을 붙임
        concat = torch.cat((embed_user_output, pooled_output), -1) # (B, hidden_size+factor_num)

        # dropout 통과 및 회귀 결과 생성
        sequence_output = self.regressor_dropout(concat)
        logits = self.regressor(sequence_output).view(-1) # (B, 1) -> (B, )
        # view(-1)하는 이유 : 길이가 B인 벡터로 변환해줌 -> 맞춰주어야 Loss 계산이 제대로 됨

        # 손실계산
        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)
        # loss 값을 출력하는 것이 forward 함수의 메인
        # logits - 출력 최종 예측값

## 설정 초기화

In [ ]:
from transformers import AutoConfig

# 사전 학습된 모델의 설정을 로드
distil_config = AutoConfig.from_pretrained(pretrained_model_tag,
                                           num_labels=1
                                          )
distil_config.user_num=len(user_map) # user_map의 길이를 통해 유저 수를 설정
distil_config.factor_num=32 # 유저 임베딩의 차원 수를 설정
distil_config.regressor_dropout=0.2 # 회귀 레이어에 적용할 dropout 확률을 설정

## 모델 로딩

In [ ]:
#huggingface에서 모델을 로딩하면서 원래 가지고 있던 정보를 가져와서 새로 가져온 정보를 덧붙여서 새로운 모델을 만드는 과정
distil_model = DistilBertForRegression.from_pretrained(pretrained_model_tag, config=distil_config)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForRegression: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForRegression from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForRegression from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForRegression were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['regressor.weight', 'regressor.bias', 'embed_user.weight']
You should probably TRAIN this model on a do

### 모델 아키텍쳐 관찰

In [ ]:
distil_model

DistilBertForRegression(
  (embed_user): Embedding(943, 32)
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(

In [ ]:
# distil_model(**batch)

## 훈련 변수 설정

In [ ]:
from transformers import TrainingArguments

num_epochs = 3
# 코랩에서 GPU 메모리 부족 에러가 나는 경우 batch_size를 16,32 등으로 줄여보자
batch_size = 64
logging_steps = len(train_ds) // batch_size
model_name = f"distilbert-fine-tuned"
training_args = TrainingArguments(
    output_dir=model_name, # 훈련된 모델과 관련된 출력을 저장할 디렉토리
    log_level="error", # 로깅 수준을 "error"로 설정
    num_train_epochs=num_epochs, # 훈련할 에포크 수
    per_device_train_batch_size=batch_size, # 훈련 시 GPU/CPU당 배치 크기
    per_device_eval_batch_size=batch_size, evaluation_strategy="epoch",
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
    logging_steps=logging_steps, push_to_hub=False) # 허브에 푸시하지 않음

## 메트릭 정의 (MAE)

In [ ]:
from sklearn.metrics import mean_absolute_error #사이킷런의 모듈 사용

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()  # if necessary
    return {'mae': mean_absolute_error(labels, predictions)}

## 트레이너 생성

In [ ]:
from transformers import Trainer #객체 Trainer 생성

trainer = Trainer(model=distil_model, args=training_args,
                  data_collator=data_collator, compute_metrics=compute_metrics,
                  train_dataset=train_ds,
                  eval_dataset=validation_ds,
                  tokenizer=tokenizer)

In [ ]:
distil_model.config.use_cache = False

# 훈련 실시

수행시간
- Colab T4: 1시간30분
- RTX 4090: 8분 30초
- Titan RTX: 27분 10초

In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mae
1,1.145300,1.106835,0.825970
2,1.043200,1.070631,0.835418
3,1.011700,1.050189,0.818166


## 훈련 결과 저장

In [ ]:
trainer.save_model()

## 예측

In [ ]:
predictions = trainer.predict(validation_ds)


In [ ]:
sorted(zip(predictions.predictions, validation_ds['label'].numpy()), key=lambda x: x[0])[:100]

[(1.93655, 1.0),
 (1.9821336, 1.0),
 (1.9849739, 2.0),
 (1.9849998, 1.0),
 (1.9904329, 1.0),
 (1.9966578, 1.0),
 (2.0057275, 1.0),
 (2.009785, 1.0),
 (2.0218205, 1.0),
 (2.0222242, 1.0),
 (2.0224144, 1.0),
 (2.036059, 1.0),
 (2.037121, 3.0),
 (2.0471208, 1.0),
 (2.053591, 1.0),
 (2.0598319, 4.0),
 (2.0677955, 1.0),
 (2.0692992, 1.0),
 (2.070652, 1.0),
 (2.0706737, 1.0),
 (2.0781267, 1.0),
 (2.0845087, 1.0),
 (2.085183, 1.0),
 (2.0854301, 1.0),
 (2.088692, 1.0),
 (2.0899732, 1.0),
 (2.0900285, 2.0),
 (2.0916436, 1.0),
 (2.09253, 2.0),
 (2.0956066, 1.0),
 (2.0972092, 1.0),
 (2.0983198, 1.0),
 (2.0995638, 2.0),
 (2.100936, 1.0),
 (2.1015584, 1.0),
 (2.1030004, 3.0),
 (2.1039765, 1.0),
 (2.1052363, 1.0),
 (2.109249, 1.0),
 (2.1108098, 4.0),
 (2.1110983, 1.0),
 (2.111775, 2.0),
 (2.1136553, 3.0),
 (2.1143215, 1.0),
 (2.1152937, 1.0),
 (2.1172042, 3.0),
 (2.117749, 1.0),
 (2.1202047, 1.0),
 (2.1210632, 3.0),
 (2.1223161, 5.0),
 (2.1232505, 2.0),
 (2.1259606, 3.0),
 (2.1266644, 2.0),
 (2.1308

In [ ]:
sorted(zip(predictions.predictions, validation_ds['label'].numpy()), key=lambda x: x[0])[-100:]
#(prediction, 정답 label) 형태로 출력

[(4.56685, 4.0),
 (4.566903, 4.0),
 (4.567014, 5.0),
 (4.567132, 5.0),
 (4.5684533, 5.0),
 (4.568764, 5.0),
 (4.568792, 5.0),
 (4.56909, 4.0),
 (4.569403, 5.0),
 (4.569482, 4.0),
 (4.5696793, 4.0),
 (4.569756, 5.0),
 (4.5702796, 5.0),
 (4.5704074, 4.0),
 (4.5705514, 5.0),
 (4.5711956, 5.0),
 (4.571484, 5.0),
 (4.5720816, 5.0),
 (4.572104, 4.0),
 (4.5727153, 4.0),
 (4.57297, 5.0),
 (4.5731015, 5.0),
 (4.573111, 5.0),
 (4.5734806, 4.0),
 (4.5736294, 5.0),
 (4.573857, 4.0),
 (4.5738587, 5.0),
 (4.573892, 4.0),
 (4.5755525, 5.0),
 (4.5756617, 5.0),
 (4.5760036, 5.0),
 (4.576281, 5.0),
 (4.5769806, 5.0),
 (4.577007, 5.0),
 (4.577288, 3.0),
 (4.5776005, 4.0),
 (4.578296, 5.0),
 (4.578387, 5.0),
 (4.5787396, 3.0),
 (4.578815, 5.0),
 (4.5792613, 5.0),
 (4.579449, 3.0),
 (4.5796285, 5.0),
 (4.5796976, 3.0),
 (4.5805907, 5.0),
 (4.5807824, 4.0),
 (4.5809965, 5.0),
 (4.5811505, 5.0),
 (4.581184, 2.0),
 (4.581283, 4.0),
 (4.581406, 5.0),
 (4.581631, 5.0),
 (4.58192, 5.0),
 (4.5820813, 5.0),
 (4.58